In [1]:
import bitalino
import numpy as np
import time
import pygame
import math

# Adresse MAC du capteur
macAddress = "98:D3:11:FE:03:67"

# Tentative de connexion au capteur BITalino
try:
    device = bitalino.BITalino(macAddress)
    time.sleep(1)
    print("Capteur BITalino connecté")
except Exception as e:
    print(f"Erreur de connexion au capteur: {e}")
    exit()

# Paramètres de lecture
srate = 1000
nframes = 100

# Démarrage de l'acquisition des données
try:
    device.start(srate, [0, 1])  # Lire les broches 0 (EDA) et 1 (ECG)
    print("START")
except Exception as e:
    print(f"Erreur lors du démarrage de l'acquisition: {e}")
    exit()

# Initialisation de Pygame
pygame.init()

# Définir la taille de la fenêtre
window_size = (1100, 700)
screen = pygame.display.set_mode(window_size)
pygame.display.set_caption("Données EDA et ECG - Interface Futuriste")

# Police pour afficher les données
font = pygame.font.SysFont("Arial", 30)
hud_font = pygame.font.SysFont("Press Start 2P", 25)

# Couleurs
BACKGROUND_COLOR = (20, 20, 20)  # Fond sombre
TEXT_COLOR = (0, 255, 0)  # Texte vert fluo
HIGHLIGHT_COLOR = (255, 0, 0)  # Alerte rouge
NEON_BLUE = (0, 255, 255)  # Bleu néon
NEON_PURPLE = (255, 0, 255)  # Violet néon
ECG_COLOR = (255, 255, 0)  # Jaune pour l'ECG

# Variables pour stocker les dernières valeurs EDA et ECG
last_EDA_value = 0.0
last_ECG_value = 0.0

# Liste pour stocker les valeurs ECG pour le tracé
ecg_values = []

# Fonction pour dessiner le fond
def draw_background():
    screen.fill(BACKGROUND_COLOR)

# Fonction d'animation d'un cercle dynamique pour EDA
def draw_dynamic_circle(EDA_value):
    radius = int(20 + (EDA_value / 100) * 30)  # Le rayon change en fonction de la valeur EDA
    pygame.draw.circle(screen, NEON_PURPLE, (window_size[0] - 300, 200), radius)
    pygame.draw.circle(screen, (0, 0, 0), (window_size[0] - 300, 200), radius - 10)

# Afficher l'interface HUD de manière élégante
def draw_hud(EDA_value, ECG_value):
    # Titre
    title_text = hud_font.render("CAPTEUR EDA et ECG", True, TEXT_COLOR)
    screen.blit(title_text, (30, 30))

    # Valeur EDA actuelle
    EDA_text = font.render(f"EDA: {EDA_value:.2f}", True, TEXT_COLOR)
    screen.blit(EDA_text, (30, 100))

    # Valeur ECG actuelle
    ECG_text = font.render(f"ECG: {ECG_value:.2f}", True, ECG_COLOR)
    screen.blit(ECG_text, (30, window_size[1] - 200))

# Fonction pour dessiner le tracé de l'ECG
def draw_ecg_trace(ecg_values):
    if len(ecg_values) < 2:
        return

    # Définir la zone de tracé de l'ECG
    ecg_area_height = 1 # Hauteur de la zone de tracé de l'ECG
    ecg_area_top = window_size[1] - 50  # Position verticale du haut de la zone de tracé

    # Tracer les valeurs ECG dans la zone définie
    scale_factor = 10  # Facteur d'échelle pour ajuster la hauteur de la courbe
    for i in range(1, len(ecg_values)):
        x1 = (i - 1) * (window_size[0] / len(ecg_values))
        y1 = ecg_area_top + ecg_area_height / 2 - ecg_values[i - 1] / scale_factor
        x2 = i * (window_size[0] / len(ecg_values))
        y2 = ecg_area_top + ecg_area_height / 2 - ecg_values[i] / scale_factor
        pygame.draw.line(screen, ECG_COLOR, (x1, y1), (x2, y2), 2)
# Boucle principale pour afficher les données en temps réel
running = True
while running:
    # Gérer les événements Pygame (fermeture de la fenêtre, etc.)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    try:
        # Lire les données du capteur
        data = device.read(nframes)

        # Si la moyenne est inférieure à 1, on arrête la lecture
        if np.mean(data[:, 1]) < 1:
            break

        EDA = data[:, -2]
        ECG = data[:, -1]
        last_EDA_value = EDA[-1]  # Dernière valeur EDA
        last_ECG_value = ECG[-1]  # Dernière valeur ECG

        # Ajouter la dernière valeur ECG à la liste pour le tracé
        ecg_values.append(last_ECG_value)
        if len(ecg_values) > window_size[0]:
            ecg_values.pop(0)  # Limiter la taille de la liste

    except Exception as e:
        print(f"Erreur de lecture des données : {e}")
        break

    # Dessiner le fond, les cercles dynamiques et l'interface
    draw_background()
    draw_dynamic_circle(last_EDA_value)
    draw_hud(last_EDA_value, last_ECG_value)
    draw_ecg_trace(ecg_values)

    # Mettre à jour l'affichage
    pygame.display.flip()

    # Attendre un peu avant de relire les données
    pygame.time.delay(100)

# Arrêter le capteur et fermer Pygame proprement
print("STOP")
device.trigger([0, 0])
device.stop()
device.close()

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
Capteur BITalino connecté
START
STOP
